In [1]:
import os
import gc
import pandas as pd
import numpy as np
from datetime import datetime
import csv

import warnings
warnings.filterwarnings("ignore")

# Split Test.csv file for easier processing

In [ ]:
def split_csv(filehandler, delimiter=',', row_limit=150000,
          output_name_template='test_%s.csv', output_path='.', keep_headers=True):
    reader = csv.reader(filehandler, delimiter=delimiter)
    current_piece = 1
    current_out_path = os.path.join(
        output_path,
        output_name_template % current_piece
    )
    current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
    current_limit = row_limit
    if keep_headers:
        headers = next(reader)
        current_out_writer.writerow(headers)
    for i, row in enumerate(reader):
        if i + 1 > current_limit:
            current_piece += 1
            current_limit = row_limit * current_piece
            current_out_path = os.path.join(
                output_path,
                output_name_template % current_piece
            )
            current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
            if keep_headers:
                current_out_writer.writerow(headers)
        current_out_writer.writerow(row)

In [ ]:
split_csv(open('input/test.csv', 'r'),row_limit=148000,output_path='input/test_split')

In [2]:
def processing(df_train):
    df_train['click_time']=pd.to_datetime(df_train['click_time']) #convert the click_time 
    df_train['dayofweek'] = df_train['click_time'].dt.dayofweek.astype('uint8')
    df_train['date'] = df_train['click_time'].dt.dayofyear.astype('uint8')
    df_train['hour']=df_train['click_time'].dt.hour.astype('uint8')
    df_train['minute']=df_train['click_time'].dt.minute.astype('uint8')
    df_train['second']= df_train['click_time'].dt.second.astype('uint8')
    df_train.loc[:,'time_segment'] = 2
    df_train.loc[df_train.loc[:,'hour']<17,'time_segment'] = 1
    df_train.loc[df_train['hour']<14,'time_segment'] = 0
    df_train.loc[df_train['hour']<6,'time_segment'] = 3
    df_train.loc[df_train['hour']<2,'time_segment'] = 2
    total_sum = len(df_train)
    df_train['app_count'] = df_train.groupby('app')['app'].transform('count')
    df_train.loc[:,'app_segment'] = 3
    df_train.loc[df_train.loc[:,'app_count']<total_sum*.05,'app_segment'] = 2
    df_train.loc[df_train.loc[:,'app_count']<total_sum*.01,'app_segment'] = 1
    df_train.loc[df_train.loc[:,'app_count']<total_sum*.005,'app_segment'] = 0
    df_train= df_train.drop(['app_count'], axis=1)
    df_train['channel_count'] = df_train.groupby('channel')['channel'].transform('count')
    df_train.loc[:,'channel_segment'] = 4
    df_train.loc[df_train.loc[:,'channel_count']<total_sum*.08,'channel_segment'] = 3
    df_train.loc[df_train.loc[:,'channel_count']<total_sum*.03,'channel_segment'] = 2
    df_train.loc[df_train.loc[:,'channel_count']<total_sum*.01,'channel_segment'] = 1
    df_train.loc[df_train.loc[:,'channel_count']<total_sum*.005,'channel_segment'] = 0
    df_train= df_train.drop(['channel_count'], axis=1)
    df_train['os_count'] = df_train.groupby('os')['os'].transform('count')
    df_train.loc[:,'os_segment'] = 4
    df_train.loc[df_train.loc[:,'os_count']<total_sum*.23,'os_segment'] = 3
    df_train.loc[df_train.loc[:,'os_count']<total_sum*.2,'os_segment'] = 2
    df_train.loc[df_train.loc[:,'os_count']<total_sum*.04,'os_segment'] = 1
    df_train.loc[df_train.loc[:,'os_count']<total_sum*.02,'os_segment'] = 0
    df_train= df_train.drop(['os_count'], axis=1)
    df_train['device_count'] = df_train.groupby('device')['device'].transform('count')
    df_train.loc[:,'device_segment'] = 2
    df_train.loc[df_train.loc[:,'device_count']<total_sum*.1,'device_segment'] = 1
    df_train.loc[df_train.loc[:,'device_count']<total_sum*.04,'device_segment'] = 0
    df_train= df_train.drop(['device_count'], axis=1)

    # Define all the groupby transformations
    GROUPBY_AGGREGATIONS = [

        # V1 - GroupBy Features #
        #########################    
        # Variance in hour, for ip-app-os
        {'groupby': ['ip','app','os'], 'select': 'hour', 'agg': 'var'},
        # Variance in date, for ip-app-os
        {'groupby': ['ip','app','os'], 'select': 'date', 'agg': 'var'},
        # Count, for ip-app
        {'groupby': ['ip', 'app'], 'select': 'channel', 'agg': 'count'},        
        # Count, for ip-app-os
        {'groupby': ['ip', 'app', 'os'], 'select': 'channel', 'agg': 'count'},
        # Mean hour, for ip-app-channel
        {'groupby': ['ip','app','channel'], 'select': 'hour', 'agg': 'mean'}, 
        # Mean date, for ip-app-channel
        {'groupby': ['ip','app','channel'], 'select': 'date', 'agg': 'mean'}, 


        # V2 - GroupBy Features #
        #########################
        # Average clicks on app by distinct users; is it an app they return to?
        {'groupby': ['app'], 
         'select': 'ip', 
         'agg': lambda x: float(len(x)) / len(x.unique()), 
         'agg_name': 'AvgViewPerDistinct'
        },
        # How popular is the app or channel?
        {'groupby': ['app'], 'select': 'channel', 'agg': 'count'},
        {'groupby': ['channel'], 'select': 'app', 'agg': 'count'},

        # V3 - GroupBy Features                                              #
        # https://www.kaggle.com/bk0000/non-blending-lightgbm-model-lb-0-977 #
        ###################################################################### 
        {'groupby': ['ip'], 'select': 'channel', 'agg': 'nunique'}, 
        {'groupby': ['ip'], 'select': 'app', 'agg': 'nunique'}, 
        {'groupby': ['ip','app'], 'select': 'os', 'agg': 'nunique'}, 
        {'groupby': ['ip'], 'select': 'device', 'agg': 'nunique'}, 
        {'groupby': ['app'], 'select': 'channel', 'agg': 'nunique'}, 
        {'groupby': ['ip', 'device', 'os'], 'select': 'app', 'agg': 'nunique'}, 
        {'groupby': ['ip','device','os'], 'select': 'app', 'agg': 'cumcount'}, 
        {'groupby': ['ip'], 'select': 'app', 'agg': 'cumcount'}, 
        {'groupby': ['ip'], 'select': 'os', 'agg': 'cumcount'}    
    ]

    # Apply all the groupby transformations
    for spec in GROUPBY_AGGREGATIONS:

        # Name of the aggregation we're applying
        agg_name = spec['agg_name'] if 'agg_name' in spec else spec['agg']

        # Name of new feature
        new_feature = '{}_{}_{}'.format('_'.join(spec['groupby']), agg_name, spec['select'])

        # Unique list of features to select
        all_features = list(set(spec['groupby'] + [spec['select']]))

        # Perform the groupby
        gp = df_train[all_features]. \
            groupby(spec['groupby'])[spec['select']]. \
            agg(spec['agg']). \
            reset_index(). \
            rename(index=str, columns={spec['select']: new_feature})

        # Merge back to X_total
        if 'cumcount' == spec['agg']:
            df_train[new_feature] = gp[0].values
        else:
            df_train = df_train.merge(gp, on=spec['groupby'], how='left')

         # Clear memory
        del gp
        gc.collect()

    GROUP_BY_NEXT_CLICKS = [

        # V1
        {'groupby': ['ip']},
        {'groupby': ['ip', 'app']},
        {'groupby': ['ip', 'channel']},
        {'groupby': ['ip', 'os']},

        # V3
        {'groupby': ['ip', 'app', 'device', 'os', 'channel']},
        {'groupby': ['ip', 'os', 'device']},
        {'groupby': ['ip', 'os', 'device', 'app']}
    ]

    # Calculate the time to next click for each group
    for spec in GROUP_BY_NEXT_CLICKS:

        # Name of new feature
        new_feature = '{}_nextClick'.format('_'.join(spec['groupby']))    

        # Unique list of features to select
        all_features = spec['groupby'] + ['click_time']

        # Run calculation
        df_train[new_feature] = df_train[all_features].groupby(spec['groupby']).click_time.transform(lambda x: x.diff().shift(-1)).dt.seconds
        gc.collect()

    HISTORY_CLICKS = {
        'identical_clicks': ['ip', 'app', 'device', 'os', 'channel'],
        'app_clicks': ['ip', 'app']
    }

    # Go through different group-by combinations
    for fname, fset in HISTORY_CLICKS.items():

        # Clicks in the past
        df_train['prev_'+fname] = df_train. \
            groupby(fset). \
            cumcount(). \
            rename('prev_'+fname)

        # Clicks in the future
        df_train['future_'+fname] = df_train.iloc[::-1]. \
            groupby(fset). \
            cumcount(). \
            rename('future_'+fname).iloc[::-1]

        gc.collect()

    df_train = df_train.drop(columns=['click_time','date',
                                      'os','channel','device','app','ip'])
    return df_train

In [3]:
traincolumns = ['ip','app', 'device', 'os', 'channel', 'click_time']
dtypes = {
        'ip'            : 'uint32',
        'app'           : 'uint16',
        'device'        : 'uint16',
        'os'            : 'uint16',
        'channel'       : 'uint16',
        'click_id'      : 'uint32'
        }


In [4]:
import glob
path = "input/test_split/*.csv"
for fname in glob.glob(path):
    print('Reading ' + fname)
    df_train = pd.read_csv(fname,usecols = traincolumns, dtype=dtypes, header=0)
    print('Processing '+ fname)
    df_train = processing(df_train)
    print('Writing '+ fname[:-4]+'fe.csv')
    df_train.to_csv(fname[:-4]+'fe.csv')
    gc.collect()

Reading input/test_split/test_97.csv
Processing input/test_split/test_97.csv
Writing input/test_split/test_97fe.csv
Reading input/test_split/test_67.csv
Processing input/test_split/test_67.csv
Writing input/test_split/test_67fe.csv
Reading input/test_split/test_126.csv
Processing input/test_split/test_126.csv
Writing input/test_split/test_126fe.csv
Reading input/test_split/test_13.csv
Processing input/test_split/test_13.csv
Writing input/test_split/test_13fe.csv
Reading input/test_split/test_111.csv
Processing input/test_split/test_111.csv
Writing input/test_split/test_111fe.csv
Reading input/test_split/test_1.csv
Processing input/test_split/test_1.csv
Writing input/test_split/test_1fe.csv
Reading input/test_split/test_68.csv
Processing input/test_split/test_68.csv
Writing input/test_split/test_68fe.csv
Reading input/test_split/test_78.csv
Processing input/test_split/test_78.csv
Writing input/test_split/test_78fe.csv
Reading input/test_split/test_127.csv
Processing input/test_split/tes

Writing input/test_split/test_114fe.csv
Reading input/test_split/test_20.csv
Processing input/test_split/test_20.csv
Writing input/test_split/test_20fe.csv
Reading input/test_split/test_42.csv
Processing input/test_split/test_42.csv
Writing input/test_split/test_42fe.csv
Reading input/test_split/test_89.csv
Processing input/test_split/test_89.csv
Writing input/test_split/test_89fe.csv
Reading input/test_split/test_110.csv
Processing input/test_split/test_110.csv
Writing input/test_split/test_110fe.csv
Reading input/test_split/test_37.csv
Processing input/test_split/test_37.csv
Writing input/test_split/test_37fe.csv
Reading input/test_split/test_50.csv
Processing input/test_split/test_50.csv
Writing input/test_split/test_50fe.csv
Reading input/test_split/test_26.csv
Processing input/test_split/test_26.csv
Writing input/test_split/test_26fe.csv
Reading input/test_split/test_82.csv
Processing input/test_split/test_82.csv
Writing input/test_split/test_82fe.csv
Reading input/test_split/test